**Team Members: Ethan Wong, Timmy Ren, Mason Shu, Medha Nalamada, Carson Mullen, Bethel Kim**

**Morning Cohort (11 AM - 1 PM)**

*Note to all: Please pull any changes from the repo before working on this file!*

In [9]:
# Install Libraries
#!pip install deep-translator
#!pip install langdetect
#!pip install vaderSentiment

In [18]:
# Import Libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    WebDriverException,
)
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import numpy as np
import json
from deep_translator import GoogleTranslator
from langdetect import detect
from collections import Counter
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer #type: ignore

# Task A: Scrape from ratebeer.com - extract 5-6K reviews

In [ ]:
def handle_cookie_consent():
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "onetrust-banner-sdk"))
        )
        accept_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@id='onetrust-accept-btn-handler']"))
        )
        accept_button.click()
        WebDriverWait(driver, 5).until(
            EC.invisibility_of_element_located((By.ID, "onetrust-banner-sdk"))
        )
    except (TimeoutException, NoSuchElementException, WebDriverException):
        pass

def handle_ratebeer_banner():
    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'RateBeerBestBanner___StyledDiv-jAauHc'))
        )
        close_button = driver.find_element(By.CSS_SELECTOR, '.RateBeerBestBanner___StyledDiv-jAauHc .MuiIconButton-root')
        close_button.click()
        WebDriverWait(driver, 5).until(
            EC.invisibility_of_element_located((By.CLASS_NAME, 'RateBeerBestBanner___StyledDiv-jAauHc'))
        )
    except (TimeoutException, NoSuchElementException, WebDriverException):
        pass

json_file_path = 'beer_reviews.json'
with open(json_file_path, 'w', encoding='utf-8') as f:
    f.write('[')

chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), options=chrome_options
)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

# Navigate to the RateBeer Top Beers page
driver.get("https://www.ratebeer.com/top-beers")

handle_cookie_consent()

WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located(
        (By.XPATH, "//div[contains(@class,'DataTable__Row')]/div[2]/a")
    )
)

# Get the list of beers and their links
beer_elements = driver.find_elements(By.XPATH, "//div[contains(@class,'DataTable__Row')]/div[2]/a")
beers = []
for beer_element in beer_elements:
    full_text = beer_element.text
    product_name = full_text.split('\n')[0]  # Take the first line before the newline
    product_link = beer_element.get_attribute('href')
    beers.append({'name': product_name, 'link': product_link})

total_reviews_collected = 0
max_reviews = 30000  # Set your desired number of reviews to collect
first_review = True  # Flag to handle commas in JSON

# Loop through all beers
for i in range(len(beers)):
    product_name = beers[i]['name']
    product_link = beers[i]['link']

    # Navigate to the beer page
    driver.get(product_link)

    handle_cookie_consent()
    handle_ratebeer_banner()

    # Wait for the dropdown to be present
    try:
        dropdown = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "MuiSelect-selectMenu"))
        )
        dropdown.click()

        # Wait for the dropdown options to be visible
        option_100 = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//li[contains(text(), '100')]"))
        )
        option_100.click()
    except (TimeoutException, NoSuchElementException, WebDriverException):
        continue

    # Get the total number of reviews
    try:
        num_items_text = driver.find_element(By.CSS_SELECTOR, ".MuiTablePagination-selectRoot + .MuiTypography-colorInherit").text
        total_reviews = int(num_items_text.split(" ")[-1])
        num_pages = int(np.ceil(total_reviews / 100))
    except (NoSuchElementException, ValueError, WebDriverException):
        continue

    for j in range(num_pages):
        time.sleep(2)
        # Expand all "Show more" buttons
        while True:
            try:
                show_more_button = driver.find_element(By.XPATH, "//span[text()='Show more']")
                driver.execute_script("arguments[0].scrollIntoView(true);", show_more_button)
                show_more_button.click()
                time.sleep(1)
            except NoSuchElementException:
                break
            except ElementClickInterceptedException:
                handle_ratebeer_banner()
                driver.execute_script("arguments[0].click();", show_more_button)
            except Exception as e:
                print(f"Could not click 'Show more' button: {e}")
                break

        # Get all comments and store as a list
        comments_elements = driver.find_elements(By.CSS_SELECTOR, ".pre-wrap.MuiTypography-body1")
        product_reviews = [comment.text for comment in comments_elements]

        rating_elements = driver.find_elements(By.CSS_SELECTOR, ".bRPQdN.MuiTypography-subtitle1")
        ratings = [rating.text for rating in rating_elements]

        beer_data_list = []

        for review, rating in zip(product_reviews, ratings):
            beer_data = {
                'product_name': product_name,
                'product_review': review,
                'rating': rating
            }
            beer_data_list.append(beer_data)
            total_reviews_collected += 1

            if total_reviews_collected >= max_reviews:
                break

        # Write data to the JSON file
        with open(json_file_path, 'a', encoding='utf-8') as f:
            for beer_data in beer_data_list:
                if not first_review:
                    f.write(',\n')
                json.dump(beer_data, f, ensure_ascii=False)
                first_review = False
        beer_data_list = []

        if total_reviews_collected >= max_reviews:
            break

        if j + 1 < num_pages:
            try:
                button_to_click = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, ".MuiIconButton-colorInherit + .MuiIconButton-colorInherit"))
                )
                handle_ratebeer_banner()
                button_to_click.click()
                time.sleep(1)
            except ElementClickInterceptedException:
                handle_ratebeer_banner()
                driver.execute_script("arguments[0].click();", button_to_click)
            except Exception as e:
                print(f"Could not click 'Next' button: {e}")
                break

    if total_reviews_collected >= max_reviews:
        break

# Close the JSON array properly
with open(json_file_path, 'a', encoding='utf-8') as f:
    f.write(']')

driver.quit()

In [3]:
# Load the JSON data into a DataFrame
with open('beer_reviews.json', 'r', encoding='utf-8') as file:
    beer_reviews = json.load(file)

df = pd.DataFrame(beer_reviews)
df

,product_name,product_review,rating
0,Toppling Goliath Kentucky Brunch,"You need personal informations from companies,...",3.3
1,Toppling Goliath Kentucky Brunch,"Bottle after MBCC 2024. Black colour, malty ar...",4.5
2,Toppling Goliath Kentucky Brunch,Thank you for sharing this Chris - Black with ...,4.3
3,Toppling Goliath Kentucky Brunch,"Boxed beer at home, proper glassware. Pitch bl...",4.7
4,Toppling Goliath Kentucky Brunch,"From backlog. (As 2018 Vintage) 0,3 litre Bott...",4.9
...,...,...,...
29371,Superstition Blue Berry White,"Bottle 82/100. Pours a deep, inky purple. More...",4.8
29372,Superstition Blue Berry White,Pours a deep berry hue. Aroma of rich blueberr...,4.8
29373,Superstition Blue Berry White,"If you look at my reviews, you will see how hi...",5.0
29374,Superstition Blue Berry White,Huge thank you to Dakine for sharing this with...,4.6


In [4]:
# Downsampling Reviews
freq = pd.DataFrame(df.value_counts('product_name')).reset_index()
freq.columns = ['product_name', 'count']
filtered_df = df[df['product_name'].isin(freq[freq['count'] > 200]['product_name'])]

In [5]:
sampled_df = filtered_df.groupby('product_name', group_keys=False).apply(lambda x: x.sample(min(len(x), 100)))

C:\Users\ethan\AppData\Local\Temp\ipykernel_26184\1693224999.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = filtered_df.groupby('product_name', group_keys=False).apply(lambda x: x.sample(min(len(x), 100)))


In [6]:
sampled_df

,product_name,product_review,rating
8864,3 Fonteinen J & J Oude Geuze Blauw,Blind tasting: Amber gold. Ornate tiny-bubbled...,4.6
8890,3 Fonteinen J & J Oude Geuze Blauw,75 cL bottle. Pours hazy orange with a huge wh...,4.2
8942,3 Fonteinen J & J Oude Geuze Blauw,(labelled as Drie Fonteinen Akkurats Gold Geuz...,3.9
8727,3 Fonteinen J & J Oude Geuze Blauw,Hazy golden of color with a small white head. ...,3.8
8873,3 Fonteinen J & J Oude Geuze Blauw,fucking wow. Thanks to JohnC for breaking this...,4.5
...,...,...,...
4847,Westvleteren 12,Very very smooth. Best before 2006. Alcohol i...,4.3
2079,Westvleteren 12,Tis is definately a great beer. As so manybeer...,3.8
3310,Westvleteren 12,Vintage 2003. Poured from 12 oz bottle into ...,4.7
3382,Westvleteren 12,"Ölen är ganska stärk i doften, alkoholprocente...",4.3


In [7]:
sampled_df.value_counts('product_name')

product_name
3 Fonteinen J & J Oude Geuze Blauw                          100
3 Fonteinen Oude Geuze 1998 (50th Anniversary)              100
Trappistes Rochefort 10                                     100
Toppling Goliath Mornin' Delight                            100
Three Floyds Dark Lord - Bourbon Vanilla Bean               100
Three Floyds Dark Lord - Bourbon Barrel Aged                100
Russian River Pliny the Younger                             100
Perennial Abraxas - Barrel-Aged                             100
Närke Kaggen Stormaktsporter                                100
La Face Cachée de la Pomme Neige Récolte d'Hiver            100
Goose Island Bourbon County Stout - Rare 2010               100
Goose Island Bourbon County Stout - Proprietor's 2013       100
Founders KBS (Kentucky Breakfast Stout)                     100
Founders CBS (Canadian Breakfast Stout)                     100
De Dolle Stille Nacht Reserva 2000                          100
Cigar City Hunahpu's Imperi

In [8]:
#sampled_df.to_csv('sampled_beer_review.csv')

In [9]:
# Function to translate text to English if it's not already in English
def translate_to_english_if_needed(text):
    try:
        # Detect the language of the text
        lang = detect(text)
        # Translate only if the language is not English
        if lang != 'en':
            translated = GoogleTranslator(source='auto', target='en').translate(text)
            return translated
        else:
            return text
    except Exception as e:
        return str(e)

# Apply the translation function to the 'product_review' column
sampled_df['product_review'] = sampled_df['product_review'].apply(translate_to_english_if_needed)

In [11]:
# Export the DataFrame to a CSV or json file to prevent re-running the code in the future
#sampled_df.to_csv('final_beer_reviews.csv', index=False)

# Task B: Perform a word frequency analysis to find the most important beer attributes and ensure that the attributes specified are likely to be mentioned together in a review

In [13]:
df = pd.read_csv('final_beer_reviews.csv')

(2200, 3)

In [14]:
# Choose three attributes we identify in this task to be used for subsequent tasks! 

#Load final reviews dataframe 
df = pd.read_csv('final_beer_reviews.csv')

#Word frequency analysis of entire df 
df['product_review'] = df['product_review'].apply(str)
all_text = ' '.join(df['product_review'].dropna())
words = re.findall(r'\w+', all_text.lower())
word_counts = Counter(words)
word_freq_df = pd.DataFrame(word_counts.items(), columns=['Word', 'Frequency']).sort_values(by='Frequency', ascending=False)

#Define a common list of attributes to search through 
attributes = [
    'aggressive', 'balanced', 'crisp', 'sweet', 'diacetyl', 'estery', 'floral', 'fruity', 'malty', 
    'robust', 'light', 'hoppy', 'bitter', 'clove', 'dry', 'earthy', 'herbal', 'nutty', 'piney', 
    'spicy', 'tart', 'woody', 'creamy', 'toast', 'smooth', 'oaky',
    'peppery', 'zesty', 'citrusy', 'pungent', 'sour', 'smoky', 'fragrant', 'lingering',
    'thin', 'yeasty', 'burnt', 'rich', 'mellow', 'clean', 'dank', 
    'warming',
]

attribute_freq = word_freq_df[word_freq_df['Word'].isin(attributes)]
top_attributes = attribute_freq.sort_values(by='Frequency', ascending=False)
top_10_attributes = top_attributes[:10]['Word']
print('Top 10 Attributes Chosen:\n', top_10_attributes)

Top 10 Attributes Chosen:
 227       sweet
196       light
457      smooth
212        rich
835      bitter
220      creamy
536    balanced
172         dry
177        sour
770        thin
Name: Word, dtype: object


In [15]:
#Performing a lift analysis to assess their associations within a message
from itertools import combinations
from collections import defaultdict

# Top 3 attributes
input_attributes = ['sweet', 'light', 'smooth']

def clean_text(text):
    text = re.sub(r'\W+', ' ', text.lower())  # Remove punctuation and convert to lowercase
    words = text.split()
    return words 

def find_attribute_pairs(message, attributes, window=50):
    message = re.sub(r'\W+', ' ', message.lower())  # Clean the message
    words = message.split()
    attribute_indices = {attribute: [] for attribute in attributes}

    # Track the position of each attribute in the message
    for i, word in enumerate(words):
        if word in attributes:
            attribute_indices[word].append(i)

    # Find pairs within the given window
    attribute_pairs = set()
    attribute_list = list(attribute_indices.keys())
    for idx1, attribute1 in enumerate(attribute_list):
        indices1 = attribute_indices[attribute1]
        for idx2 in range(idx1 + 1, len(attribute_list)):
            attribute2 = attribute_list[idx2]
            indices2 = attribute_indices[attribute2]
            for i1 in indices1:
                for i2 in indices2:
                    if abs(i1 - i2) - 1 <= window:
                        pair = tuple(sorted((attribute1, attribute2)))
                        attribute_pairs.add(pair)
    return attribute_pairs

# Function to calculate word frequencies and word pair co-occurrences
def calculate_frequencies(df, input_attributes, window=50):
    word_frequency = defaultdict(int)
    word_pair_frequency = defaultdict(int)

    # Loop through each post and count word occurrences
    for index, row in df.iterrows():
        message = row['product_review']
        words_in_message = set(clean_text(message))
        attributes_in_message = words_in_message.intersection(input_attributes)
        
        # Update word frequencies
        for attribute in attributes_in_message:
            word_frequency[attribute] += 1

        # Update pair frequencies
        attribute_pairs = find_attribute_pairs(message, input_attributes, window=window)
        for attribute_pair in attribute_pairs:
            if attribute_pair[0] in attributes_in_message and attribute_pair[1] in attributes_in_message:
                word_pair_frequency[attribute_pair] += 1

    return word_frequency, word_pair_frequency


# Function to calculate lift
def calculate_lift(word_frequency, word_pair_frequency, total_posts):
    lift_values = []

    for (attribute1, attribute2), pair_count in word_pair_frequency.items():
        p_attribute1 = word_frequency[attribute1] / total_posts
        p_attribute2 = word_frequency[attribute2] / total_posts
        p_both = pair_count / total_posts

        if p_attribute1 * p_attribute2 > 0:
            lift = p_both / (p_attribute1 * p_attribute2)
            lift_values.append({
                'Attribute 1': attribute1,
                'Attribute 2': attribute2,
                'Lift': lift
            })

    return pd.DataFrame(lift_values)

# Main function to process data and compute lift ratios
def main(df, top_10_brands_list, window=50):
    # Step 1: Calculate word and word pair frequencies
    total_posts = len(df)
    word_frequency, word_pair_frequency = calculate_frequencies(df, input_attributes, window)
    
    # Step 2: Calculate lift values
    lift_df = calculate_lift(word_frequency, word_pair_frequency, total_posts)
    
    # Step 3: Create lift matrix
    lift_matrix = lift_df.pivot(index='Attribute 1', columns='Attribute 2', values='Lift')
    
    # Ensure all attributes are present in both axes
    all_attributes = sorted(set(input_attributes))
    lift_matrix = lift_matrix.reindex(index=all_attributes, columns=all_attributes)
    
    # Replace missing values with 0
    lift_matrix = lift_matrix.fillna(0)
    
    # Convert DataFrame to object type to allow setting string values
    lift_matrix = lift_matrix.astype(object)
    
    # Set the diagonal and lower triangle to '-'
    for i in range(len(all_attributes)):
        lift_matrix.iloc[i, i] = '-'  # Set diagonal to '-'
        for j in range(i):
            lift_matrix.iloc[i, j] = '-'  # Set lower triangle to '-'
    
    # Remove labels from being displayed
    lift_matrix.index.name = None
    lift_matrix.columns.name = None
    
    # Return both lift_matrix and lift_df
    return lift_matrix, lift_df

# Execute the main function and capture the lift matrix and lift DataFrame
lift_matrix, lift_df = main(df, input_attributes, window=5000)

# Display the lift matrix as a DataFrame
lift_matrix

,light,smooth,sweet
light,-,1.19684,1.129274
smooth,-,-,1.09851
sweet,-,-,-


# Task C: Perform a similarity analysis using cosine similarity with the three attributes specified by the customer and the reviews

**Note: Used the bag-of-words model as opposed to word embeddings**

In [19]:
# Script should accept a file as input that has the product attributes and calculate similarity scores (0-1) 
    # between these attributes and each review
# Output file should have three columns: product_name (will have a row for each review), product_review, 
    # and similarity_score

# Task D: Perform a sentiment analysis for every review

In [20]:
# Viewing sentiment scores for each word in the reviews to determine if alterations are needed

analyzer = SentimentIntensityAnalyzer()

all_reviews = ' '.join(df['product_review'].tolist())
words = set(all_reviews.split())

word_scores = {}

for word in words:
    if word in analyzer.lexicon:
        word_scores[word] = analyzer.lexicon[word]
    else:
        word_scores[word] = 'Not in VADER lexicon'

word_scores_df = pd.DataFrame(list(word_scores.items()), columns=['Word', 'Sentiment Score'])

,Word,Sentiment Score
0,memorial,Not in VADER lexicon
1,spirits,Not in VADER lexicon
2,"lacing,",Not in VADER lexicon
3,somehow,Not in VADER lexicon
4,imagine,Not in VADER lexicon
...,...,...
17428,products.,Not in VADER lexicon
17429,coutesy,Not in VADER lexicon
17430,(what?!),Not in VADER lexicon
17431,piney-grapefruity,Not in VADER lexicon


In [24]:
# Print out the sentiment scores for each word
filtered_df = word_scores_df[word_scores_df['Sentiment Score'] != 'Not in VADER lexicon']

for index, row in filtered_df.iterrows():
    print(f"Word: {row['Word']}, Sentiment Score: {row['Sentiment Score']}")

Word: easily, Sentiment Score: 1.4
Word: hiding, Sentiment Score: -1.2
Word: retained, Sentiment Score: 0.1
Word: leave, Sentiment Score: -0.2
Word: stop, Sentiment Score: -1.2
Word: arguably, Sentiment Score: -1.0
Word: screwed, Sentiment Score: -2.2
Word: true, Sentiment Score: 1.8
Word: impressive, Sentiment Score: 2.3
Word: smile, Sentiment Score: 1.5
Word: healthy, Sentiment Score: 1.7
Word: noble, Sentiment Score: 2.0
Word: proudly, Sentiment Score: 2.6
Word: ruins, Sentiment Score: -1.9
Word: struck, Sentiment Score: -1.0
Word: ease, Sentiment Score: 1.5
Word: risk, Sentiment Score: -1.1
Word: excels, Sentiment Score: 2.5
Word: dominates, Sentiment Score: 0.2
Word: nuts, Sentiment Score: -1.3
Word: bitch, Sentiment Score: -2.8
Word: impresses, Sentiment Score: 2.1
Word: brisk, Sentiment Score: 0.6
Word: fine, Sentiment Score: 0.8
Word: great, Sentiment Score: 3.1
Word: fatiguing, Sentiment Score: -1.2
Word: giving, Sentiment Score: 1.4
Word: definitely, Sentiment Score: 1.7
Word

Based on the these scores, it might be best to adjust terms related to bitterness, light, complex and add terms like sour and aggressive!

In [25]:
new_words = {
    'bitter': 0.5,  # Adjusted from negative to slightly positive for IPAs
    'bitterness': 0.5,
    'light': 1.0,  # Adjusted to reflect a more neutral-positive trait
    'low': 0.5,  # Adjusted to neutral for certain beer styles
    'strong': 2.5,  # Slight increase for beer context
    'sour': 0.8,  # Positive adjustment for sour beer styles
    'complex': 2.0,  # Higher for describing flavor complexity
    'aggressive': 0.5,  # Neutralized in the context of bold beer flavors
}
analyzer.lexicon.update(new_words)

In [26]:
#Calculating sentiment for each review

analyzer = SentimentIntensityAnalyzer()

def get_sentiment_vader(review):
    sentiment_scores = analyzer.polarity_scores(review)
    return sentiment_scores['compound']  

# Apply the VADER sentiment analysis to each product review
df['sentiment_score'] = df['product_review'].apply(get_sentiment_vader)

In [30]:
df[df['sentiment_score'] < 0]

,product_name,product_review,rating,sentiment_score
13,3 Fonteinen J & J Oude Geuze Blauw,Bottle. Cloudy golden colour with a little hea...,3.1,-0.0516
28,3 Fonteinen J & J Oude Geuze Blauw,"Bottle thanks to mes, i havnt rated in a week ...",4.9,-0.2890
41,3 Fonteinen J & J Oude Geuze Blauw,"Aroma/Malty-light,\nAppearance Head/Initial-sm...",1.9,-0.3412
67,3 Fonteinen J & J Oude Geuze Blauw,Bottle at Max & Jess’s for an impromptu barbec...,3.9,-0.4404
85,3 Fonteinen J & J Oude Geuze Blauw,Way worse than druiven. I also don’t like the ...,1.3,-0.1531
...,...,...,...,...
2146,Westvleteren 12,Pours a slightly hazy dark brown with a small ...,4.5,-0.6006
2154,Westvleteren 12,"bread, nuts, chocolate, coffee, ??? Why in Eng...",4.7,-0.2406
2159,Westvleteren 12,"Dark, reddish brown colour. Thick and creamy h...",4.2,-0.4215
2178,Westvleteren 12,"bread, resin, dough, alcohol, raisin, amber,ha...",4.3,-0.3182


# Task E: Create an evaluation score for each beer that uses both similarity and sentiment scores and recommended three products to the customer

In [21]:
# Add sentiment and similarity scores for the three products recommended.

# Task F: How would our recommendations change if we use word vectors instead of the standard bag-of-words cosine similarity?

In [22]:
# Consider the % of reviews that mention a preferred attribute
    # For a recommended product, what % of its reviews mention an attribute specified by the customer
# Differences between bag-of-words and word vector approaches
# Bag of words, under certain conditions, will likely be better than word embeddings for recommender systems
# Show rating, similarity scores, and sentiments for the products recommended in this task and the previous one

# Task G: How would our recommendations differ if we ignored the similarity and feature sentiment scores and just chose the 3 highest rated products fro the entire data set? Would these products meet the requirements of the user looking for recommendations? Why or why not?

In [23]:
# Justify with analysis - use similarity and sentiment scores as well as overall ratings to answer this questions

# Task H: From 10 beers in the data, choose 1, and find the most similar beer among the remaining 9

In [24]:
# Show logic we are using in addition to finding the most similar product with markdown; explain method/logic